#### Ensuring that all the required libraries are installed

In [ ]:
pip install psycopg2 pandas numpy sklearn searborn nltk emoji matplotlib json os time re strings

#### Import the required libraries

In [ ]:
import psycopg2 as psy
import json
import os
import time

In [ ]:
from emoji import UNICODE_EMOJI

#### Establish connection with the PostgreSQL database

In [ ]:
database_host = "localhost"
database_name = "dbl"
database_user = "kcinnay"
database_pass = "DBL01"

In [ ]:
con = psy.connect(
    host = database_host,
    database = database_name,
    user = database_user,
    password = database_pass
)
cur = con.cursor()

#### Create a table in the database to store the tweet data in

In [ ]:
cur.execute("DROP TABLE IF EXISTS tweets")

In [ ]:
cur.execute("""
    CREATE TABLE tweets
        (
            created_at TIMESTAMPTZ DEFAULT NULL,
            timestamp_ms TEXT DEFAULT NULL,

            id_str TEXT PRIMARY KEY,

            user_id_str TEXT DEFAULT NULL,
            user_name TEXT DEFAULT NULL,
            user_created_at TIMESTAMPTZ DEFAULT NULL,
            user_followers INT DEFAULT NULL,
            user_statuses INT DEFAULT NULL,
            user_favourites INT DEFAULT NULL,

            full_text TEXT DEFAULT NULL,

            in_reply_to_status_id_str TEXT DEFAULT NULL,
            in_reply_to_user_id_str TEXT DEFAULT NULL,
            in_reply_to_screen_name TEXT DEFAULT NULL,

            quoted_status_id_str TEXT DEFAULT NULL,

            contains_emoji BOOLEAN DEFAULT NULL
        )
""")

#### Attributes of a tweet object
'created_at',  
'id',  
'id_str',  
'text',  
'source',  
'truncated',  
'in_reply_to_status_id',  
'in_reply_to_status_id_str',  
'in_reply_to_user_id',  
'in_reply_to_user_id_str',  
'in_reply_to_screen_name',  
'user',  
'geo',  
'coordinates',  
'place',  
'contributors',  
'retweeted_status',  
'is_quote_status',  
'quote_count',  
'reply_count',  
'retweet_count',  
'favorite_count',  
'entities',  
'favorited',  
'retweeted',  
'filter_level',  
'lang',  
'timestamp_ms'

#### Database columns and their datatype
**created_at** TIMESTAMPTZ DEFAULT NULL  
**timestamp_ms** TEXT DEFAULT NULL  
 
**id_str** TEXT DEFAULT NULL  
 
**user_id_str** TEXT DEFAULT NULL  
**user_name** TEXT DEFAULT NULL  
**user_created_at** TIMESTAMPTZ DEFAULT NULL  
**user_followers** INT DEFAULT NULL   
**user_statuses** INT DEFAULT NULL   
**user_favourites** INT DEFAULT NULL  
 
**full_text** TEXT DEFAULT NULL  
 
**in_reply_to_status_id_str** TEXT DEFAULT NULL  
**in_reply_to_user_id_str** TEXT DEFAULT NULL  
**in_reply_to_screen_name** TEXT DEFAULT NULL  
 
**quoted_status_id_str** TEXT DEFAULT NULL  
 
**contains_emoji** BOOLEAN DEFAULT NULL  

#### Initializing the error counters

In [ ]:
errors = {
    "total_tweets": 0,
    "empty": 0,
    "json": 0,
    "filter": 0,
    "duplicate": 0
}

#### This function checks if a tweet is parseable

In [ ]:
def tweet_parseable(tweet): 
    if not tweet.strip(): 
        errors["empty"] += 1
        return False
    try: 
        return json.loads(tweet)
    except: 
        errors["json"] += 1
        return False

#### Functions that return values which can be inserted into the database

In [ ]:
def created_at(tweet): 
    return tweet["created_at"]
def timestamp_ms(tweet): 
    return tweet["timestamp_ms"]

def id_str(tweet): 
    return tweet["id_str"]

def user_id_str(tweet): 
    return tweet["user"]["id_str"]
def user_name(tweet): 
    return tweet["user"]["name"]
def user_created_at(tweet): 
    return tweet["user"]["created_at"]
def user_followers(tweet): 
    return tweet["user"]["followers_count"]
def user_statuses(tweet): 
    return tweet["user"]["statuses_count"]
def user_favourites(tweet): 
    return tweet["user"]["favourites_count"]

def full_text(tweet): 
    try: 
        return tweet["extended_tweet"]["full_text"]
    except: 
        return tweet["text"]
    
def in_reply_to_status_id_str(tweet): 
    return tweet["in_reply_to_status_id_str"]
def in_reply_to_user_id_str(tweet): 
    return tweet["in_reply_to_user_id_str"]
def in_reply_to_screen_name(tweet): 
    return tweet["in_reply_to_screen_name"]

def quoted_status_id_str(tweet): 
    try: 
        return tweet["quoted_status_id_str"]
    except: 
        return None

def contains_emoji(tweet): 
    for letter in full_text(tweet): 
        if letter in UNICODE_EMOJI["en"]: 
            return True
    return False

#### Filter the tweet based on certain criteria

In [ ]:
def is_delete(tweet): 
    return "delete" in tweet.keys()

def is_retweet(tweet): 
    try: 
        tweet["retweeted_status"]
        return True
    except: 
        return False

def is_not_english(tweet): 
    if tweet["lang"] == "en": 
        return False
    return True

def has_no_emojis(tweet): 
    if tweet["lang"] == "und": 
        if contains_emoji(tweet): 
            return False
    return True

In [ ]:
def filter_tweet(tweet): 
    if is_delete(tweet): 
        errors["filter"] += 1
        return False
    if is_retweet(tweet): 
        errors["filter"] += 1
        return False
    if is_not_english(tweet): 
        if has_no_emojis(tweet): 
            errors["filter"] += 1
            return False
    return True

#### Loads a given data array with values into the database

In [ ]:
def insert_into_database(data): 
    try: 
        cur.execute("""INSERT INTO tweets (
            created_at,
            timestamp_ms,

            id_str,

            user_id_str,
            user_name,
            user_created_at,
            user_followers,
            user_statuses,
            user_favourites,

            full_text,

            in_reply_to_status_id_str,
            in_reply_to_user_id_str,
            in_reply_to_screen_name,

            quoted_status_id_str,

            contains_emoji
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""", (data[0],data[1],data[2],data[3],data[4],data[5],data[6],data[7],data[8],data[9],data[10],data[11],data[12],data[13],data[14]))
        con.commit()
    except Exception as e: 
        con.commit()
        errors["duplicate"] += 1

#### This reads all of the files in the folder called data, and subsequently extracts all the tweets and their values by using the above helper functions

In [ ]:
files = os.listdir("data")

for idx, file in enumerate(files): 
    print("Processing file "+str(idx+1)+" / "+str(len(files))+"...")
    start = time.time()
    with open("data/"+file, 'rb') as f: 
        for tweetline in f.readlines(): 
            errors["total_tweets"] += 1
            tweet = tweet_parseable(tweetline)
            if tweet: 
                if filter_tweet(tweet): 
                    data = [
                        created_at(tweet),
                        timestamp_ms(tweet),

                        id_str(tweet),

                        user_id_str(tweet),
                        user_name(tweet),
                        user_created_at(tweet),
                        user_followers(tweet),
                        user_statuses(tweet),
                        user_favourites(tweet),

                        full_text(tweet),

                        in_reply_to_status_id_str(tweet),
                        in_reply_to_user_id_str(tweet),
                        in_reply_to_screen_name(tweet),

                        quoted_status_id_str(tweet),

                        contains_emoji(tweet)
                    ]
                    insert_into_database(data)
        end = time.time()
        print("Finished in "+str(end-start)+" seconds")
        print(errors)

#### Close connection

In [ ]:
cur.close()
con.close()